In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
from pkg.controller.robot_config import RobotType
from pkg.utils.test_scripts import *
from pkg.planning.filtering.filter_interface import *
from pkg.planning.constraint.constraint_common import BindingTransform

ROBOT_TYPE = RobotType.panda
VISUALIZE = True

ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, HOME_POSE, GRIP_DEPTH = get_single_robot_params(ROBOT_TYPE)
s_builder, pscene = prepare_single_robot_scene(ROBOT_TYPE, ROBOT_NAME, TOOL_LINK, TOOL_XYZ, TOOL_RPY, VISUALIZE=VISUALIZE)
crob, gscene = pscene.combined_robot, pscene.gscene

connection command:
panda0: False
Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


## check LatticedChecker scene

In [2]:
from pkg.planning.filtering.grasp_filter import GraspChecker
gcheck = GraspChecker(pscene)

from pkg.planning.filtering.latticized_filter import LatticedChecker
lcheck = LatticedChecker(pscene, gcheck=gcheck)

cpath = os.path.join(SCENE_PATH, "LatticedChecker")
files = sorted(os.listdir(cpath))
print("File count= {}".format(len(files)))

from pkg.planning.filtering.latticized_filter import ARM_SHAPE, GRASP_SHAPE, \
    WDH_GRASP, L_CELL_GRASP, OFFSET_ZERO_GRASP, WDH_ARM, L_CELL_ARM, OFFSET_ZERO_ARM
from pkg.planning.filtering.lattice_model.latticizer_py import get_centers

centers_grasp = get_centers(GRASP_SHAPE, L_CELL_GRASP, OFFSET_ZERO_GRASP)
centers_arm = get_centers(ARM_SHAPE, L_CELL_ARM, OFFSET_ZERO_ARM)

assert len(pscene.robot_chain_dict.values())==1, "not single robot scene"
shoulder_height = lcheck.shoulder_height_dict[ROBOT_NAME]

File count= 899


In [3]:
file_path = os.path.join(cpath, files[0])
print("file_path: {}".format(file_path))
scene_data = load_pickle(file_path)
state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)
res = lcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
print("result/log: {} / {} ({})".format(res, result[1]>0.5, np.round(result[1], 2)))

file_path: /home/rnb/Projects/rnb-planning/data/checker_scenes/LatticedChecker/00000012-OK.pkl
result/log: True / True (0.91)


In [4]:
gtimer = GlobalTimer.instance()
gtimer.reset()
res_saved_list = []
res_infer_list = []
for fname in files[:100]:
    file_path = os.path.join(cpath, fname)
    scene_data = load_pickle(file_path)
    state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)
    with gtimer.block("test"):
        res = lcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
    res_saved_list.append(result[1]>0.5)
    res_infer_list.append(res)

In [5]:
print("="*50)
print_confusion_mat(GT=res_saved_list, Res=res_infer_list)
print("="*50)
print(gtimer)
print("="*50)

           PP         PN        100
GP         82          2     97.62%
GN          8          8     50.00%
AL     91.11%     80.00%     90.00%
test: 	2163.7 ms/100 = 21.6 ms (16.201/35.16)
set_relation: 	5.3 ms/100 = 0.1 ms (0.048/0.085)
effector: 	167.2 ms/100 = 1.7 ms (1.247/2.814)
arm: 	118.3 ms/100 = 1.2 ms (0.795/1.746)
indexing_vertices: 	65.8 ms/100 = 0.7 ms (0.405/1.112)
query_wait_response: 	1780.3 ms/100 = 17.8 ms (12.723/32.161)



In [6]:
# lcheck.query_quit_dict["panda"][0] = True

### check differences

In [8]:
for fname in files:
    print("========== {} ===========".format(fname))
    file_path = os.path.join(cpath, fname)
    scene_data = load_pickle(file_path)
    state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)
    res = lcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
    grasp_img = lcheck.grasp_img_p_dict[ROBOT_TYPE.name][0]
    grasp_tool_img = grasp_img[:,:,:,0]
    grasp_obj_img = grasp_img[:,:,:,1]
    grasp_tar_img = grasp_img[:,:,:,2]
    arm_img = lcheck.arm_img_p_dict[ROBOT_TYPE.name][0][:,:,:,0]
    rh_vals = lcheck.rh_vals_p_dict[ROBOT_TYPE.name][0]

    grasp_tool_idx = np.where(grasp_tool_img)
    grasp_obj_idx = np.where(grasp_obj_img)
    grasp_tar_idx = np.where(grasp_tar_img)
    arm_tar_idx = np.where(arm_img)
    radius, theta, height = lcheck.rth_last
    offset_ee = cyl2cart(radius, theta, height)


    res_strict = lcheck.check_strict(btf=btf, Q_dict=Q_dict, **kwargs)
    grasp_img = lcheck.grasp_img_p_dict[ROBOT_TYPE.name][0]
    grasp_tool_img = grasp_img[:,:,:,0]
    grasp_obj_img = grasp_img[:,:,:,1]
    grasp_tar_img = grasp_img[:,:,:,2]
    arm_img = lcheck.arm_img_p_dict[ROBOT_TYPE.name][0][:,:,:,0]
    rh_vals = lcheck.rh_vals_p_dict[ROBOT_TYPE.name][0]

    grasp_tool_idx_strict = np.where(grasp_tool_img)
    grasp_obj_idx_strict = np.where(grasp_obj_img)
    grasp_tar_idx_strict = np.where(grasp_tar_img)
    arm_tar_idx_strict = np.where(arm_img)
    radius, theta, height = lcheck.rth_last
    offset_ee = cyl2cart(radius, theta, height)

    grasp_tool_idx_strict = set([tuple(grid) for grid in np.array(grasp_tool_idx_strict).transpose()])
    grasp_obj_idx_strict = set([tuple(grid) for grid in np.array(grasp_obj_idx_strict).transpose()])
    grasp_tar_idx_strict = set([tuple(grid) for grid in np.array(grasp_tar_idx_strict).transpose()])
    arm_tar_idx_strict = set([tuple(grid) for grid in np.array(arm_tar_idx_strict).transpose()])

    grasp_tool_idx = set([tuple(grid) for grid in np.array(grasp_tool_idx).transpose()])
    grasp_obj_idx = set([tuple(grid) for grid in np.array(grasp_obj_idx).transpose()])
    grasp_tar_idx = set([tuple(grid) for grid in np.array(grasp_tar_idx).transpose()])
    arm_tar_idx = set([tuple(grid) for grid in np.array(arm_tar_idx).transpose()])

    if len(list(grasp_tool_idx - grasp_tool_idx_strict )) > 0: 
        print("tool cell over {} / {}".format(len(list(grasp_tool_idx - grasp_tool_idx_strict)), len(grasp_tool_idx_strict)))
    if len(list(grasp_obj_idx - grasp_obj_idx_strict )) > 0: 
        print("obj cell over {} / {}".format(len(list(grasp_obj_idx - grasp_obj_idx_strict)), len(grasp_obj_idx_strict)))
    if len(list(grasp_tar_idx - grasp_tar_idx_strict )) > 0: 
        print("tar cell over {} / {}".format(len(list(grasp_tar_idx - grasp_tar_idx_strict)), len(grasp_tar_idx_strict)))
    if len(list(arm_tar_idx -arm_tar_idx_strict )) > 0: 
        print("arm cell over {} / {}".format(len(list(arm_tar_idx -arm_tar_idx_strict)), len(arm_tar_idx_strict)))

    tool_idx_diff = tuple(np.transpose(sorted(grasp_tool_idx_strict - grasp_tool_idx)))
    obj_idx_diff =tuple(np.transpose( sorted(grasp_obj_idx_strict - grasp_obj_idx)))
    tar_idx_diff = tuple(np.transpose(sorted(grasp_tar_idx_strict - grasp_tar_idx)))
    arm_idx_diff = tuple(np.transpose(sorted(arm_tar_idx_strict - arm_tar_idx)))

    if float(len(tool_idx_diff))/len(grasp_tool_idx_strict) > 0.1: 
        print("tool diff too much {}".format(round(
            float(len(tool_idx_diff))/len(grasp_tool_idx_strict), 3)))
    if float(len(obj_idx_diff))/len(grasp_obj_idx_strict) > 0.2: 
        print("obj diff too much {}".format(round(
            float(len(obj_idx_diff))/len(grasp_obj_idx_strict), 3)))
    if float(len(tar_idx_diff))/len(grasp_tar_idx_strict) > 0.1: 
        print("tar diff too much {}".format(round(
            float(len(tar_idx_diff))/len(grasp_tar_idx_strict), 3)))
    if float(len(arm_idx_diff))/len(arm_tar_idx_strict) > 0.1:
        print("arm diff too much {}".format(round(
            float(len(arm_idx_diff))/len(arm_tar_idx_strict), 3)))

========== 00000012-OK.pkl ===========
========== 00000013-OK.pkl ===========
========== 00000014-OK.pkl ===========
========== 00000015-OK.pkl ===========
========== 00000016-OK.pkl ===========
========== 00000017-OK.pkl ===========
========== 00000018-OK.pkl ===========
========== 00000019-OK.pkl ===========
========== 00000020-OK.pkl ===========
========== 00000021-OK.pkl ===========
========== 00000022-OK.pkl ===========
========== 00000023-OK.pkl ===========
========== 00000024-OK.pkl ===========
========== 00000025-OK.pkl ===========
========== 00000026-OK.pkl ===========
========== 00000027-OK.pkl ===========
========== 00000028-OK.pkl ===========
========== 00000029-OK.pkl ===========
========== 00000030-OK.pkl ===========
========== 00000031-OK.pkl ===========
========== 00000032-OK.pkl ===========
========== 00000033-OK.pkl ===========
obj cell over 1 / 19
========== 00000034-OK.pkl ===========
========== 00000035-OK.pkl ===========
========== 00000036-OK.pkl ===========
====

========== 00000204-OK.pkl ===========
========== 00000205-OK.pkl ===========
========== 00000206-OK.pkl ===========
========== 00000207-OK.pkl ===========
========== 00000208-OK.pkl ===========
========== 00000209-OK.pkl ===========
========== 00000210-OK.pkl ===========
========== 00000211-OK.pkl ===========
========== 00000212-OK.pkl ===========
========== 00000213-OK.pkl ===========
========== 00000214-OK.pkl ===========
========== 00000215-OK.pkl ===========
========== 00000216-OK.pkl ===========
========== 00000217-OK.pkl ===========
========== 00000218-OK.pkl ===========
arm cell over 1 / 1360
========== 00000219-OK.pkl ===========
========== 00000220-OK.pkl ===========
========== 00000221-OK.pkl ===========
========== 00000222-OK.pkl ===========
========== 00000223-OK.pkl ===========
========== 00000224-OK.pkl ===========
tar cell over 1 / 1143
========== 00000225-OK.pkl ===========
========== 00000226-OK.pkl ===========
========== 00000227-OK.pkl ===========
========== 0000022

tar cell over 2 / 1678
========== 00000391-OK.pkl ===========
arm cell over 2 / 1875
========== 00000392-OK.pkl ===========
========== 00000393-OK.pkl ===========
========== 00000394-OK.pkl ===========
arm cell over 1 / 1888
========== 00000395-OK.pkl ===========
arm cell over 1 / 1912
========== 00000396-OK.pkl ===========
========== 00000397-OK.pkl ===========
========== 00000398-OK.pkl ===========
========== 00000399-OK.pkl ===========
========== 00000400-OK.pkl ===========
========== 00000401-OK.pkl ===========
tar cell over 1 / 1677
========== 00000402-OK.pkl ===========
========== 00000403-OK.pkl ===========
========== 00000404-OK.pkl ===========
arm cell over 1 / 1873
========== 00000405-OK.pkl ===========
========== 00000406-OK.pkl ===========
========== 00000407-OK.pkl ===========
tar cell over 2 / 1801
========== 00000408-OK.pkl ===========
========== 00000409-OK.pkl ===========
arm cell over 2 / 1871
========== 00000410-OK.pkl ===========
========== 00000411-OK.pkl =========

========== 00000573-OK.pkl ===========
arm cell over 1 / 1917
========== 00000574-OK.pkl ===========
tar cell over 1 / 1545
========== 00000575-OK.pkl ===========
========== 00000576-OK.pkl ===========
========== 00000577-OK.pkl ===========
========== 00000578-OK.pkl ===========
========== 00000579-OK.pkl ===========
========== 00000580-OK.pkl ===========
========== 00000581-OK.pkl ===========
arm cell over 1 / 1912
========== 00000582-OK.pkl ===========
========== 00000583-OK.pkl ===========
obj cell over 1 / 19
arm cell over 1 / 1929
========== 00000584-OK.pkl ===========
tar cell over 1 / 1644
========== 00000585-OK.pkl ===========
========== 00000586-OK.pkl ===========
========== 00000587-OK.pkl ===========
========== 00000588-OK.pkl ===========
========== 00000589-OK.pkl ===========
========== 00000590-OK.pkl ===========
========== 00000591-OK.pkl ===========
========== 00000592-OK.pkl ===========
========== 00000593-OK.pkl ===========
========== 00000594-OK.pkl ===========
======

========== 00000762-OK.pkl ===========
========== 00000763-OK.pkl ===========
arm cell over 1 / 1876
========== 00000764-OK.pkl ===========
========== 00000765-OK.pkl ===========
========== 00000766-OK.pkl ===========
tar cell over 2 / 1607
========== 00000767-OK.pkl ===========
========== 00000768-OK.pkl ===========
========== 00000769-OK.pkl ===========
========== 00000770-OK.pkl ===========
arm cell over 1 / 1859
========== 00000771-OK.pkl ===========
arm cell over 1 / 1878
========== 00000772-OK.pkl ===========
========== 00000773-OK.pkl ===========
========== 00000774-OK.pkl ===========
========== 00000775-OK.pkl ===========
========== 00000776-OK.pkl ===========
========== 00000777-OK.pkl ===========
========== 00000778-OK.pkl ===========
========== 00000779-OK.pkl ===========
========== 00000780-OK.pkl ===========
tar cell over 1 / 1564
========== 00000781-OK.pkl ===========
arm cell over 2 / 1910
========== 00000782-OK.pkl ===========
========== 00000783-OK.pkl ===========
====

In [10]:
for fname in files[375:376]:
    print("========== {} ===========".format(fname))
    file_path = os.path.join(cpath, fname)
    scene_data = load_pickle(file_path)
    state, btf, Q_dict, kwargs, error_state, result = load_unpack_scene_args(pscene, scene_data)
    res = lcheck.check(btf=btf, Q_dict=Q_dict, **kwargs)
    grasp_img = lcheck.grasp_img_p_dict[ROBOT_TYPE.name][0]
    grasp_tool_img = grasp_img[:,:,:,0]
    grasp_obj_img = grasp_img[:,:,:,1]
    grasp_tar_img = grasp_img[:,:,:,2]
    arm_img = lcheck.arm_img_p_dict[ROBOT_TYPE.name][0][:,:,:,0]
    rh_vals = lcheck.rh_vals_p_dict[ROBOT_TYPE.name][0]

    grasp_tool_idx = np.where(grasp_tool_img)
    grasp_obj_idx = np.where(grasp_obj_img)
    grasp_tar_idx = np.where(grasp_tar_img)
    arm_tar_idx = np.where(arm_img)
    radius, theta, height = lcheck.rth_last
    offset_ee = cyl2cart(radius, theta, height)


    res_strict = lcheck.check_strict(btf=btf, Q_dict=Q_dict, **kwargs)
    grasp_img = lcheck.grasp_img_p_dict[ROBOT_TYPE.name][0]
    grasp_tool_img = grasp_img[:,:,:,0]
    grasp_obj_img = grasp_img[:,:,:,1]
    grasp_tar_img = grasp_img[:,:,:,2]
    arm_img = lcheck.arm_img_p_dict[ROBOT_TYPE.name][0][:,:,:,0]
    rh_vals = lcheck.rh_vals_p_dict[ROBOT_TYPE.name][0]

    grasp_tool_idx_strict = np.where(grasp_tool_img)
    grasp_obj_idx_strict = np.where(grasp_obj_img)
    grasp_tar_idx_strict = np.where(grasp_tar_img)
    arm_tar_idx_strict = np.where(arm_img)
    radius, theta, height = lcheck.rth_last
    offset_ee = cyl2cart(radius, theta, height)

    grasp_tool_idx_strict = set([tuple(grid) for grid in np.array(grasp_tool_idx_strict).transpose()])
    grasp_obj_idx_strict = set([tuple(grid) for grid in np.array(grasp_obj_idx_strict).transpose()])
    grasp_tar_idx_strict = set([tuple(grid) for grid in np.array(grasp_tar_idx_strict).transpose()])
    arm_tar_idx_strict = set([tuple(grid) for grid in np.array(arm_tar_idx_strict).transpose()])

    grasp_tool_idx = set([tuple(grid) for grid in np.array(grasp_tool_idx).transpose()])
    grasp_obj_idx = set([tuple(grid) for grid in np.array(grasp_obj_idx).transpose()])
    grasp_tar_idx = set([tuple(grid) for grid in np.array(grasp_tar_idx).transpose()])
    arm_tar_idx = set([tuple(grid) for grid in np.array(arm_tar_idx).transpose()])

    if len(list(grasp_tool_idx - grasp_tool_idx_strict )) > 0: 
        print("tool cell over {} / {}".format(len(list(grasp_tool_idx - grasp_tool_idx_strict)), len(grasp_tool_idx_strict)))
    if len(list(grasp_obj_idx - grasp_obj_idx_strict )) > 0: 
        print("obj cell over {} / {}".format(len(list(grasp_obj_idx - grasp_obj_idx_strict)), len(grasp_obj_idx_strict)))
    if len(list(grasp_tar_idx - grasp_tar_idx_strict )) > 0: 
        print("tar cell over {} / {}".format(len(list(grasp_tar_idx - grasp_tar_idx_strict)), len(grasp_tar_idx_strict)))
    if len(list(arm_tar_idx -arm_tar_idx_strict )) > 0: 
        print("arm cell over {} / {}".format(len(list(arm_tar_idx -arm_tar_idx_strict)), len(arm_tar_idx_strict)))

    tool_idx_diff = tuple(np.transpose(sorted(grasp_tool_idx_strict - grasp_tool_idx)))
    obj_idx_diff =tuple(np.transpose( sorted(grasp_obj_idx_strict - grasp_obj_idx)))
    tar_idx_diff = tuple(np.transpose(sorted(grasp_tar_idx_strict - grasp_tar_idx)))
    arm_idx_diff = tuple(np.transpose(sorted(arm_tar_idx_strict - arm_tar_idx)))

    if float(len(tool_idx_diff))/len(grasp_tool_idx_strict) > 0.15: 
        print("tool diff too much {}".format(round(
            float(len(tool_idx_diff))/len(grasp_tool_idx_strict), 3)))
    if float(len(obj_idx_diff))/len(grasp_obj_idx_strict) > 0.3: 
        print("obj diff too much {}".format(round(
            float(len(obj_idx_diff))/len(grasp_obj_idx_strict), 3)))
    if float(len(tar_idx_diff))/len(grasp_tar_idx_strict) > 0.15: 
        print("tar diff too much {}".format(round(
            float(len(tar_idx_diff))/len(grasp_tar_idx_strict), 3)))
    if float(len(arm_idx_diff))/len(arm_tar_idx_strict) > 0.15:
        print("arm diff too much {}".format(round(
            float(len(arm_idx_diff))/len(arm_tar_idx_strict), 3)))

========== 00000387-OK.pkl ===========
obj cell over 1 / 14


### show difference

In [14]:
gscene.clear_highlight()
if len(arm_idx_diff)>0:   
    for i_c, center in enumerate(centers_arm[arm_idx_diff]):
        center_rot = np.matmul(Rot_axis(3, theta), center) + [0,0,shoulder_height]
        gscene.add_highlight_axis("lcheck", "arm_%05d"%(i_c), "base_link", dims=(0.03,) * 3,
                                  center=center_rot, orientation_mat=np.identity(3),
                                  color=(0.5, 0.5, 0.5, 0.7), axis=None)

if len(tool_idx_diff)>0:    
    for i_c, center in enumerate(centers_grasp[tool_idx_diff]):
        center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
        gscene.add_highlight_axis("lcheck", "tool_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                  center=center_rot, orientation_mat=np.identity(3),
                                  color=(0, 1, 0, 0.7), axis=None)

if len(tar_idx_diff)>0:
    for i_c, center in enumerate(centers_grasp[tar_idx_diff]):
        center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
        gscene.add_highlight_axis("lcheck", "tar_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                  center=center_rot, orientation_mat=np.identity(3),
                                  color=(1, 0, 0, 0.7), axis=None)
        
if len(obj_idx_diff)>0:
    for i_c, center in enumerate(centers_grasp[obj_idx_diff]):
        center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
        gscene.add_highlight_axis("lcheck", "obj_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                                  center=center_rot, orientation_mat=np.identity(3),
                                  color=(1, 1, 0, 0.7), axis=None)
    
pscene.show_binding(btf)

In [12]:
grasp_tool_idx_disp = tuple(np.transpose(sorted(grasp_tool_idx)))
grasp_obj_idx_disp =tuple(np.transpose( sorted(grasp_obj_idx)))
grasp_tar_idx_disp = tuple(np.transpose(sorted(grasp_tar_idx)))
arm_tar_idx_disp = tuple(np.transpose(sorted(arm_tar_idx)))

In [13]:

gscene.clear_highlight()
for i_c, center in enumerate(centers_arm[arm_tar_idx_disp]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + [0,0,shoulder_height]
    gscene.add_highlight_axis("lcheck", "arm_%05d"%(i_c), "base_link", dims=(0.03,) * 3,
                              center=center_rot, orientation_mat=np.identity(3),
                              color=(0.5, 0.5, 0.5, 0.7), axis=None)

for i_c, center in enumerate(centers_grasp[grasp_tool_idx_disp]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.add_highlight_axis("lcheck", "tool_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                              center=center_rot, orientation_mat=np.identity(3),
                              color=(0, 1, 0, 0.7), axis=None)

for i_c, center in enumerate(centers_grasp[grasp_tar_idx_disp]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.add_highlight_axis("lcheck", "tar_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                              center=center_rot, orientation_mat=np.identity(3),
                              color=(1, 0, 0, 0.7), axis=None)
        
for i_c, center in enumerate(centers_grasp[grasp_obj_idx_disp]):
    center_rot = np.matmul(Rot_axis(3, theta), center) + offset_ee
    gscene.add_highlight_axis("lcheck", "obj_%05d"%(i_c), "base_link", dims=(0.02,) * 3,
                              center=center_rot, orientation_mat=np.identity(3),
                              color=(1, 1, 0, 0.7), axis=None)
    
pscene.show_binding(btf)

## Latticizer Python version slow, only for reference

In [20]:
class PyLatticzer:
    def __init__(self, WDH=(3, 3, 2), L_CELL=0.05, OFFSET_ZERO=(1.5, 1.5, 0.5)):
        self.WDH = WDH
        self.L_CELL = L_CELL
        self.OFFSET_ZERO = OFFSET_ZERO
        self.Nwdh = tuple(np.divide(WDH, L_CELL).astype(np.int))
        self.set_centers(*(self.Nwdh + (L_CELL,) + OFFSET_ZERO))
        
    def set_centers(self, Nw, Nd, Nh, L_CELL, x_offset, y_offset, z_offset):
        OFFSET_ZERO = np.array([x_offset, y_offset, z_offset]);
        self.Nw = Nw
        self.Nd = Nd
        self.Nh = Nh
        self.L_CELL = L_CELL
        x_vals = np.stack(
            [(np.arange(Nw).astype(np.float)+0.5)*L_CELL - x_offset, [0]*Nw, [0]*Nw], axis=-1)
        y_vals = np.stack(
            [[0]*Nd, (np.arange(Nd).astype(np.float)+0.5)*L_CELL - y_offset, [0]*Nd], axis=-1)
        z_vals = np.stack(
            [[0]*Nh, [0]*Nh, (np.arange(Nh).astype(np.float)+0.5)*L_CELL - z_offset], axis=-1)

        x_vals = x_vals[:, np.newaxis,np.newaxis, :]
        y_vals = y_vals[np.newaxis, :, np.newaxis, :]
        z_vals = z_vals[np.newaxis, np.newaxis, :]
        self.centers = x_vals+y_vals+z_vals
        
    def get_collision_idc(self, gtem, T_ge):
        gtimer = GlobalTimer.instance()
        with gtimer.block("centers_tf"):
            R_ge = T_ge[:3, :3]
            P_ge = T_ge[:3, 3]
            centers_tf = np.matmul(self.centers, R_ge.transpose()) + P_ge
            radii = self.L_CELL/2
            
        with gtimer.block("dists"):
            with gtimer.block("GEO_"+gtem.gtype.name):
                if gtem.gtype == GEOTYPE.PLANE:
                    dists = np.abs(centers_tf[:,:,:,2])
                elif gtem.gtype == GEOTYPE.SPHERE:
                    dists = np.linalg.norm(centers_tf, axis=-1) - gtem.dims[0]/2
                elif gtem.gtype == GEOTYPE.CYLINDER:
                    xy_dist_ = np.linalg.norm(centers_tf[:,:,:,:2], axis=-1)
                    z_dist_ = np.abs(centers_tf[:,:,:,2])
                    xy_dist = np.clip(xy_dist_ - np.linalg.norm(gtem.dims[:2])/2, a_min=0, a_max=1e5)
                    z_dist = np.clip(z_dist_ - gtem.dims[2]/2, a_min=0, a_max=1e5)
                    dists = np.linalg.norm([z_dist, xy_dist], axis=0)
                elif gtem.gtype == GEOTYPE.CAPSULE:
                    xy_dist_ = np.linalg.norm(centers_tf[:,:,:,:2], axis=-1)
                    z_dist_ = np.clip(np.abs(centers_tf[:,:,:,2])--gtem.dims[2]/2, a_min=0, a_max=1e5)
                    dists = np.linalg.norm([xy_dist_, z_dist_], axis=0) - gtem.dims[0]/2
                else: # Assume BOX default
                    if gtem.gtype != GEOTYPE.BOX:
                        TextColors.RED.println(
                            "grid collision checking not implemented for type {}. Approximating the result with box shape".format(gtem.gtype.name))
                    out_dists = np.clip(np.abs(centers_tf) - np.divide(gtem.dims, 2), a_min=0, a_max=1e5)
                    dists = np.linalg.norm(out_dists, axis=-1)
        with gtimer.block("indexing"):
            collisions = dists <= radii
            idc_cols = np.where(collisions)
        return idc_cols

In [21]:
py_lat_eff = PyLatticzer(
    WDH=WDH_GRASP, L_CELL=L_CELL_GRASP, OFFSET_ZERO=OFFSET_ZERO_GRASP)
py_lat_arm = PyLatticzer(
    WDH=WDH_ARM, L_CELL=L_CELL_ARM, OFFSET_ZERO=OFFSET_ZERO_ARM)